# Coursera Capstone - Segmenting and Clustering Neighborhoods

# Question #1 
## Web scrape the following Wikipedia page:

- https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
- obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe

### Build the dataframe as follows:
- The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
- Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
- More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
- If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.
- Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
- In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [1]:
import pandas as pd
import numpy as np
import lxml
print('Packages Loaded')

Packages Loaded


In [2]:
tor_df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M',flavor='lxml')
tor_df[0].head()

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


#### Find rows that have no Borough assigned and drop them from the dataframe

In [3]:
drop = tor_df[0].loc[tor_df[0].Borough=='Not assigned']
drop
tor_df[0].drop(drop.index,inplace=True)
tor_df[0].head()

,Postcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


#### Find rows that have no Neighborhood assigned and assign them the value of the Borough

In [4]:
no_neigh = tor_df[0].loc[tor_df[0].Neighborhood=='Not assigned']
no_neigh
tor_df[0].loc[no_neigh.index, 'Neighborhood'] = tor_df[0].loc[no_neigh.index, 'Borough']
tor_df[0]

,Postcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
9,M9A,Queen's Park,Queen's Park
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North


#### Combine Neighborhoods that have the same Borough

In [97]:
dup_post = tor_df[0].groupby('Postcode')['Neighborhood'].apply(', '.join).reset_index()    # create new dataframe that joins the neighborhood values for each duplicated postcode
dup_post.insert(1,'Borough','N/a',True)    # add missing column with blank values
df = pd.merge(dup_post,tor_df[0], on='Postcode')   # merge the two dataframes together
df.Borough_x = df.Borough_y     # copy the original borough values
df.drop(['Borough_y','Neighborhood_y'],axis=1,inplace=True)   # remove the two original columns
df.drop_duplicates(subset='Postcode',keep='first',inplace=True)    # drop duplicates but keep first unique row
df.rename(columns={'Borough_x':'Borough','Neighborhood_x':'Neighborhood'},inplace=True)    # rename columns
df.reset_index(inplace=True)
df.drop('index', axis=1,inplace=True)
df.head()

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


#### Display the shape of the final dataframe

In [79]:
df.shape

(103, 3)

# Question #2
## Use the Geocoder package or the csv file to create a dataframe

In [98]:
# set up the dataframe
df['Latitude']=0
df['Longitude']=0

In [94]:
latlong = pd.read_csv('http://cocl.us/Geospatial_data')
latlong.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [96]:
latlong.rename(columns={'Postal Code':'Postcode'},inplace=True)
final = pd.merge(df,latlong,on='Postcode',how='left')
final.head()

,Postcode,Borough,Neighborhood,Lattitude,Longitude_x,Latitude_x,Latitude_y,Longitude_y
0,M1B,Scarborough,"Rouge, Malvern",0,0,0,43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",0,0,0,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",0,0,0,43.763573,-79.188711
3,M1G,Scarborough,Woburn,0,0,0,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,0,0,0,43.773136,-79.239476
